In [1]:
from sportsbook_spider.helpers import scraper, archive, no_vig_odds
from sportsbook_spider.stats import statsNBA
from datetime import datetime, timedelta
from tqdm import tqdm
import numpy as np

In [2]:
nba_market_ids = {
    'RA': 337,
    'REB': 157,
    'AST': 151,
    'PRA': 338,
    'PA': 335,
    'PTS': 156,
    'PR': 336,
    'BLK': 152,
    'STL': 160,
    'FG3M': 162
    }
header = {'X-API-Key': 'CHi8Hy5CEE4khd46XNYL23dCFX96oUdw6qOt1Dnh'}

In [3]:
nba = statsNBA()
nba.load()
nba.update()

Getting NBA stats


100%|██████████| 27541/27541 [00:00<00:00, 776505.76it/s]


In [ ]:
players = [(k,p['DISPLAY_FIRST_LAST'],p['PLAYERCODE'].replace('_','-')) for k,p in nba.players.items()]
for id, player, code in tqdm(players, unit='player', position=1):
    tryJr = False
    tryName = False
    player_games = [game for game in nba.gamelog if game['PLAYER_ID'] == id]
    player = player.replace('alperen-sengun','alperen-sengun-c')
    for market, mid in tqdm(list(nba_market_ids.items()), unit='market', position=2):
        url = f"https://api.bettingpros.com/v3/props/analysis?include_no_line_events=false&player_slug={code}&market_id={mid}&location=ALL&sort=desc&sport=NBA&limit=1000"
        try:
            res = scraper.get(url, headers=header, max_attempts=2)
            props = [(i['event'],i['propOffer']) for i in res['analyses'] if i['propOffer']['season'] == 2022 and i['propOffer']['line']]
        except:
            if tryJr and tryName:
                tqdm.write(f"Error finding {player}, {market}")
                continue
            else:
                if not tryJr:
                    tryJr = True
                    if '-jr' in code:
                        code = code.replace('-jr','')
                    elif '-iii' in code:
                        code = code.replace('-iii','')
                    elif '-iv' in code:
                        code = code.replace('-iv','')
                    else:
                        code = code + '-jr'
                else:
                    tryName = True
                    if not code == player.lower().replace(' ','-'):
                        code = player.lower().replace(' ','-')                    
                    
                url = f"https://api.bettingpros.com/v3/props/analysis?include_no_line_events=false&player_slug={code}&market_id={mid}&location=ALL&sort=desc&sport=NBA&limit=1000"
                try:
                    res = scraper.get(url, headers=header, max_attempts=2)
                    props = [(i['event'],i['propOffer']) for i in res['analyses'] if i['propOffer']['season'] == 2022 and i['propOffer']['line']]
                except:
                    tqdm.write(f"Error finding {player}, {market}")
                    continue
        
        for event, prop in props:
            line = prop['line']
            date = (datetime.strptime(event['scheduled'],'%Y-%m-%d %H:%M:%S') - timedelta(hours=5)).strftime('%Y-%m-%d')
            odds = [prop['cost'], prop['cost_inverse']]
            if 0 in odds:
                continue
            if prop['recommendation'] == 'under':
                odds.reverse()
            odds = no_vig_odds(odds[0], odds[1])
            game = next((i for i in player_games if i['GAME_DATE'][:10] == date), None)
            if game is None:
                tqdm.write(f"Error finding {player}, {market}, {date}")
                continue
            stats = nba.get_stats_date(game, market, line)
            stats = np.append(stats, [odds[0]]*4)
            if not date in archive.archive['NBA'][market]:
                archive.archive['NBA'][market][date] = {}
            
            archive.archive['NBA'][market][date][player] = {line: stats}

In [ ]:
url = f"https://api.bettingpros.com/v3/props/analysis?include_no_line_events=false&player_slug=justin-jefferson&market_id=105&location=ALL&sort=desc&sport=NFL&limit=1000"
res = scraper.get(url, headers=header)

In [9]:
res['analyses'][-1]['propOffer']

{'event_id': 20586,
 'market_id': 105,
 'player_id': 19236,
 'line': 83.5,
 'projection': 73.4,
 'consensus': 83.5,
 'recommendation': 'under',
 'best': 15,
 'cost': -115,
 'diff': -10.1,
 'cost_inverse': -113,
 'score': 184,
 'is_push': False,
 'date': '2022-09-11',
 'season': 2022,
 'season_type': 'REG',
 'week': 1}